#### Estimator 이해 및 fit(), predict() 메서드
- 사이킷런은 ML 모델 학습을 위해서 fit()을 학습된 모델의 예측을 위해 predict() 메서드를 제공
- 분류 알고리즘을 구현한 Classifier와 회귀 알고리즘을 구현한 Regressor를 제공
- Classifier와 Regressor를 합쳐서 Estimator라고 하며 Estimator 클래스는 fit()과 predict()를 내부에서 구현

#### Evaluation 함수 및 하이퍼 파라미터 튜닝
- cross_val_score()와 같은 evaluation 함수, GridSearchCV와 같은 하이터파라미터 튜닝을 지원하는 클래스의 경우 Estimator를 인자로 받는다.
- 인자로 받은 Estimator에 대해서 cross_val_score(), GridSearchCV.fit() 함수 내에서 Estimator의 fit()과 predict()를 호출해서 평가를 하거나 하이퍼 파라미터 튜닝을 수행한다.

#### 비지도학습과 피처 추출에서 fit()
- 사이킷런에서 비지도학습인 차원 축소, 클러스터링, 피처 추출등을 구현한 클래스 역시 대부분 fit()과 transform()을 적용한다.
- 비지도학습과 피처 추출에서 fit()은 지도학습의 fit()과 같이 학습을 의미하는 것이 아니라 입력 데이터의 형태에 맞춰 데이터을 변환하기 위한 사전 구조를 맞추는 작업이다. 
- fit()으로 변환을 위한 사전 구조를 맞추면 이후 입력 데이터의 차원 변환, 클러스터링, 피처 추출 등의 실제 작업은 transform()으로 수행한다.
- fit()과 transform()을 하나로 결합한 fit_transform()도 제공한다.

[실습 과제]  

train.csv 파일로 작성한 분석용 데이터셋에 대하여 DecisionTreeClassifier를 이용하여 다음을 수행하세요.
- 학습 데이터와 검증 데이터 분리한 후 정확도 평가
- cv=5를 적용한 KFold 교차 검증 수행 후 정확도 평가
- cv=5를 적용한 cross_val_score 교차 검증 수행 후 정확도 평가
- GridSearchCV를 이용해 최적 하이퍼파라미터를 찾고 예측성능을 측정

In [14]:
import pandas as pd
tdf = pd.read_pickle('dataset/tdf.pkl')
tdf.head()

,pclass,survived,age_cat,family,female,male,town_C,town_Q,town_S
0,1,1,0,0,1,0,0,0,1
1,1,1,4,3,0,1,0,0,1
2,1,0,4,3,1,0,0,0,1
3,1,0,0,3,0,1,0,0,1
4,1,0,0,3,1,0,0,0,1


In [3]:
y_tdf = tdf['survived']
X_tdf = tdf.drop('survived',axis=1)

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tdf,y_tdf,
                                                    test_size=0.2,
                                                   random_state=11)

In [8]:
from sklearn.metrics import accuracy_score
dt_clf = DecisionTreeClassifier(random_state=11)
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print('dt 정확도:{0:.4f}'.format(accuracy_score(y_test,dt_pred)))

dt 정확도:0.7714


In [11]:
import numpy as np
from sklearn.model_selection import KFold
def exec_kfold(clf, folds=5):
# 폴드 세트 5개인 KFold 객체를 생성, 폴드 수만큼 예측 결과 저장을 위한 리스트 생성
    kfold = KFold(n_splits=folds)
    scores = []
# KFold 교차 검증 수행
    for iter_count, (train_index, test_index) in enumerate(kfold.split(tdf)):
        X_train,X_test=X_tdf.values[train_index], X_tdf.values[test_index]
        y_train,y_test=y_tdf.values[train_index], y_tdf.values[test_index]
        
        # Classifier 학습, 예측 정확도 계산
        clf.fit(X_train,y_train)
        pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test,pred)
        scores.append(accuracy)
        print("교차 검증 {0} 정확도: {1:.4f}".format(iter_count,accuracy))
    # 5eo fold에서의 평균 정확도 계산
    mean_score = np.mean(scores)
    print("평균 정확도:{0:.4f}".format(mean_score))
# exec_kfold 호출
exec_kfold(dt_clf, folds=5)

교차 검증 0 정확도: 0.7667
교차 검증 1 정확도: 0.8373
교차 검증 2 정확도: 0.7990
교차 검증 3 정확도: 0.7273
교차 검증 4 정확도: 0.7273
평균 정확도:0.7715


In [13]:
from sklearn.model_selection import GridSearchCV
parameters = {'max_depth':[2,3,5,10],
             'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}
grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_dclf.fit(X_train,y_train)
print('GCV 최적 하이퍼 파라미터:',grid_dclf.best_params_)
print('GCV 최고 정확도:{0:.4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행
dpred = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test,dpred)
print('DT 정확도: {0:.4f}'.format(accuracy))

GCV 최적 하이퍼 파라미터: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
GCV 최고 정확도:0.8074
DT 정확도: 0.7857


[과제]  랜덤포레스트를 적용하여 다음을 수행하세요.
- 학습 데이터와 검증 데이터 분리한 후 정확도 평가
- cv=5를 적용한 KFold 교차 검증 수행 후 정확도 평가
- cv=5를 적용한 cross_val_score 교차 검증 수행 후 정확도 평가
- GridSearchCV를 이용해 최적 하이퍼파라미터를 찾고 예측성능을 측정